In [2]:
#import packages
import pandas as pd
import numpy as np
import csv
from datetime import datetime
import warnings
warnings.filterwarnings("ignore")

#####-----Read all input data file name and path into dictionary-----#####  
def read_all_inputs(path_to_conf):
    df_dic = {}
    # read all inputs needed from input_data folder
    with open(path_to_conf, 'r') as conf:
        for line in conf.readlines():
            df_dic[line.strip('\n').split(':')[0]] = pd.read_csv(line.strip('\n').split(':')[1], index_col = 0)
    #output a dictionary of file names and path
    return df_dic


#####-----Get Competitiors Info-----#####  
def get_competitors_info(df_competition):
    
    # group by team(sid)
    df_competition_grouped = df_competition.groupby('sid')

    # features_multi_rows = ['members']
    list_competitors = []

    # get team info by team
    for name, group in df_competition_grouped:
        dic_to_append = {'sid': name}
        for i, row in group.iterrows():
            if (row['name'] != 'checkbox') and (row['name'] != 'members'):
                dic_to_append[row['name']] = row['value']
            elif (row['name'] != 'checkbox') and (row['name'] == 'members'):
                if row['property'] not in dic_to_append:
                    dic_to_append[row['property']] = []
                dic_to_append[row['property']].append(row['value'])
        list_competitors.append(dic_to_append)
    
    #cast list of competitors into dataframe
    df_competition = pd.DataFrame(list_competitors)
    df_competition['member_email'] = df_competition['member_email'].map(lambda x: [] if x is np.nan else x)

    # remove test accounts
    df_competition = df_competition[~df_competition['email_primary'].str.contains('test')]
    df_competition = df_competition[~df_competition['email_primary'].str.contains('rmds')]
    df_competition = df_competition[~df_competition['email_primary'].str.contains('aliciaewei@gmail.com')]

    # classify team members into 3 types(team lead,mentor,team member)
    list_competitors = []
    for i, row in df_competition.iterrows():
        dic_to_append = {'email': row['email_primary']}
        dic_to_append['name'] = ''
        dic_to_append['type'] = '3 team lead'
        list_competitors.append(dic_to_append)

        dic_to_append = {'email': row['mentor_email']}
        dic_to_append['name'] = row['mentor_name']
        dic_to_append['type'] = '1 mentor'
        list_competitors.append(dic_to_append)

        for j in range(len(row['member_email'])):
            dic_to_append = {'email': row['member_email'][j]}
            dic_to_append['name'] = row['member_name'][j]
            dic_to_append['type'] = '2 team member'
            list_competitors.append(dic_to_append)

    # Cast list_competitors into dataframe
    df_competitors = pd.DataFrame(list_competitors)
    # Keep only one row from rows with same emails 
    df_competitors = df_competitors.groupby('email').max().reset_index()
    # drop email with NaN or '' values
    df_competitors = df_competitors.dropna(subset=['email'])
    df_competitors = df_competitors[df_competitors['email'] != '']
    
    # output team member info as df_competitors
    return df_competitors,df_competition 

#####-----Add Competition Data-----#####  
def add_competition_2021(df_users,df_competition,df_competitors):
    # add the 2021q1 competition info to df_users if email in df_competitors
    df_users['competition_2021q1'] = df_users.email.isin(list(df_competitors.email))
    # add a column as number of competition 
    df_users['competition_y_n'] = df_users['competition_2021q1'] + df_users['competition_2020']

    # add 2021q1 winner info
    first_list = df_competition[df_competition['what_is_your_team_name_']=='Insight Finders']['member_email'].values.tolist()[0]
    second_list = df_competition[df_competition['what_is_your_team_name_']=='5ACES']['member_email'].values.tolist()[0]
    third_list = df_competition[df_competition['what_is_your_team_name_']=='Data Garage']['member_email'].values.tolist()[0]

    for email in first_list:
        df_users.loc[df_users[df_users['email'] == email.strip()].index,'competition_winner_place'] = 1
    for email in second_list:
        df_users.loc[df_users[df_users['email'] == email.strip()].index,'competition_winner_place'] = 2
    for email in third_list:
        df_users.loc[df_users[df_users['email'] == email.strip()].index,'competition_winner_place'] = 3
    return df_users


#####-----Add IM_Data Event Data-----##### 
def add_im_data(df_users,df_evbr):
    df_evbr = df_evbr[['email']]
    df_evbr2 = pd.DataFrame()
    dic_to_append = {}
    for email in df_evbr['email'].unique():
        dic_to_append['mail'] = email
        df_evbr2 = df_evbr2.append(dic_to_append, ignore_index=True)
        dic_to_append = {}
    df_evbr = df_evbr2

    # merge into df_users as im_data indicator
    df = df_users.merge(df_evbr, left_on='email', right_on='mail', how='left')
    df['im_data_y_n'] = df['mail']
    for index, row in df.iterrows():
        if pd.isnull(df['mail'][index]) == True:
            df['im_data_y_n'][index] = 0
        else:
            df['im_data_y_n'][index] = 1
    
    # filter columns into df_users 
    df_users = df[['uid', 'email', 'competition_y_n', 'competition_winner_place',
           'shop_txn', 'moodle_user_y_n', 'moodle_first_login_dt',
           'moodle_first_login_days_since', 'moodle_recent_login_dt',
           'moodle_recent_login_days_since', 'num_courses_completed',
           'im_data_y_n']]
    return df_users
  
    
#####-----Add Certification Data-----##### 
def add_certification(df_users,df_cert):
    # filter NaN emails
    df_cert.groupby('email').filter(lambda x: len(x) > 1)
    # count number of certificates
    df_cert = df_cert[['email', 'uid']].groupby('email').count()
    df_cert = df_cert.reset_index()
    df_cert = df_cert.rename(columns={'uid':'num_certificate'})
    
    # add num_certificate to df_users 
    df = df_users.merge(df_cert, left_on='email', right_on='email', how='left')
    counter = 1
    for i, row in df.iterrows():
        if pd.isnull(row['num_certificate']):
            df.loc[i, 'num_certificate'] = 0
        else:
            counter += 1
    
    # fill NaN with 0 and convert num_certificate into int
    df['num_certificate'].fillna(0,inplace=True)
    df['num_certificate'] = df['num_certificate'].astype(int)
    
    # filter columns into df_users
    df_users = df[['uid', 'email', 'competition_y_n', 'competition_winner_place',
       'shop_txn', 'moodle_user_y_n', 'moodle_first_login_dt',
       'moodle_first_login_days_since', 'moodle_recent_login_dt',
       'moodle_recent_login_days_since', 'num_courses_completed',
       'im_data_y_n', 'num_certificate']]
    df_users = df_users.rename(columns = {"shop_txn": "shop_num_txn"})
    return df_users


#####-----Add The Rest of The Data-----##### 
def add_rest(df_sql_expert,df_sql_pi,df_github,df_imp_detail,df_summary_bool,df_users):
    #create pi and expert list
    #noticing there are many pi and expert missing uid from sql server it might be solved by match the email with users data frame 

    # drop sql_expert emails duplicates and rows that "uid" is missing 
    df_sql_expert = df_sql_expert.drop_duplicates(subset=['email'],keep='last')
    df_sql_expert = df_sql_expert.dropna(subset=['uid'])

    #drop sql_pi emails duplicates and rows that "uid" is missing
    df_sql_pi = df_sql_pi.drop_duplicates(subset=['email'],keep='last')
    df_sql_pi = df_sql_pi.dropna(subset=['uid'])

    sql_expert_uid=[]
    for i in df_sql_expert['uid']:
        sql_expert_uid.append(int(i))

    sql_pi_uid=[]
    for i in df_sql_pi['uid']:
        sql_pi_uid.append(int(i))
        
    # Import all user data
    # The time-invariant data might need manual updates

    # # dr_impact_score_detail
    # imp_detail (updated from MySQL, see above codes)
    df_sql_pi['volunteer']=1
    # add git_hub data
    df_github = df_github[['uid','follower_number', 'languages', 'repo_number',
           'repo_name', 'star_number']]
    # Merging all of them together
    merge = pd.merge(df_imp_detail, df_sql_pi, on='uid', how='left')
    merge = pd.merge(merge, df_summary_bool, on='uid', how='left')
    merge = pd.merge(merge, df_github, on='uid', how='left')
    df_users= pd.merge(merge, df_users, on='uid', how='left')
    df_users=df_users.rename(columns={'competition_y_n':'num_of_competition','shop_num_txn':'num_shop',
                                  'im_data_y_n':'imdata','num_courses_completed':'num_courses'})
    unnamed_colname_list=[]

    # removing these unnamed rows caused by unknown reason
    for col_name in df_users.columns:
        if 'Unnamed' in col_name:
            unnamed_colname_list.append(col_name)
    for name in unnamed_colname_list:
        df_users=df_users.drop([name], axis=1)
        
    #creating the "badge" lable. give 100 to who is expert, 50 to who is PI.
    df_users['badge']=0
    for index, row in df_users.iterrows():
        for uid in sql_expert_uid:
            if uid == row['uid']:
                df_users.loc[index, 'badge'] = 1

    for index, row in df_users.iterrows():
        for uid in sql_pi_uid:
            if uid == row['uid']:
                df_users.loc[index, 'badge'] = 0.5
    # Standardize all column names
    df_users.columns = [x.lower().replace(" ", "_").replace("-","_").replace(r"/","_").replace("\\","_") \
                  .replace("$","").replace("%","").replace("#","number") for x in df_users.columns]
    return df_users

#####-----Get Score Constants-----#####
def get_Scoring_Constants(df_scoring_constants):
    # Importing Scoring Constants
    df_scoring_constants=df_scoring_constants.dropna(subset=['column_name'])

    # convert "k" and "upper" columns from str to float 
    df_scoring_constants['k']=df_scoring_constants['k'].astype('float')
    df_scoring_constants['upper']=df_scoring_constants['upper'].astype('float')
    return df_scoring_constants


#####-----Calculate Individual Impact Score-----#####
def score_calc(df_scoring_constants, weights, df_users):   
    """ This function calculates each feature using df_scoring_constants """   
    user_score=df_users['uid']
    for index, row in df_scoring_constants.iterrows():
        score=pd.DataFrame()
        column=row['column_name']
        calc_type=row['calc_type']
        upper=row['upper']
        k=row['k']
        output='score_'+str(column)
        user_col = df_users[column]


        if column.startswith('score'):
            score=user_col
            user_score = pd.concat([user_score, user_col], axis=1, sort=False)
            
            # Adding raw columns for already calculated columns 

            #column_num=column.replace('score_','num_')
            column_num=row['raw']
            #print(column)

            user_score = pd.concat([user_score,df_users[column_num]],axis=1, sort=False)
            
        elif calc_type == 'log':
            z=-k*user_col
            #print(z)
            z = z.astype(float)
            score[output] = 200/(1+np.exp(z))-100
            user_score = pd.concat([user_score, score], axis=1, sort=False)
            user_score = pd.concat([user_score, user_col], axis=1, sort=False)
        elif calc_type == ' 100-log':
            score[output] = 200/(1+np.exp(-k*user_col))-100
            score[output] = 100-score[output]
            user_score = pd.concat([user_score, score], axis=1, sort=False)
            user_score = pd.concat([user_score, user_col], axis=1, sort=False)
        elif calc_type == 'linear':
            score[output] = user_col.apply(lambda x: (100*x/upper) if (100*x/upper) < 100 else 100)
            user_score = pd.concat([user_score, score], axis=1, sort=False)
            user_score = pd.concat([user_score, user_col], axis=1, sort=False)
        elif calc_type == 'linear_two_below':
            score[output] = user_col.apply(lambda x: round((100/3)*x-200/3,3) if (x>=2) else (0))
            user_score = pd.concat([user_score, score], axis=1, sort=False)
            user_score = pd.concat([user_score, user_col], axis=1, sort=False)
        elif calc_type == 'bolean':
            # change the bolean
            score[output] = user_col.apply(lambda x: x*100)
            #print(score[output])
            user_score = pd.concat([user_score, score], axis=1, sort=False)
            user_score = pd.concat([user_score, user_col], axis=1, sort=False)
    
    # combining all the scores into total scores.
    df_final_score=user_score

    score_cols = [col for col in df_final_score.columns if 'score' in col]
    # print(len(score_cols))
    score_cols = ['uid'] + score_cols
    df_final_score = df_final_score[score_cols]

    df_final_score = df_final_score.fillna(0)

    proj_quality_dict = dict()
    proj_engagement_dict = dict()
    proj_outcome_dict = dict()
    perceived_capacity_dict = dict()
    perceived_participation_dict = dict()
    total_score_dict = dict()

    dimension_to_metric_dict = dict()

    dimension_weight_dict = dict()
    metric_weight_dict = dict()

    # use the wieght table from our database

    for row in weights.values:
        dimension_weight_dict[row[1]] = float(row[2])
        metric_weight_dict[row[0]] = float(row[3])
        if row[1] in dimension_to_metric_dict:
            dimension_to_metric_dict[row[1]].append(row[0])
        else:
            dimension_to_metric_dict[row[1]] = []
            dimension_to_metric_dict[row[1]].append(row[0])

    #print(dimension_to_metric_dict)
    for dimension in dimension_to_metric_dict:
        df_final_score[dimension] = np.zeros(len(df_final_score))

    for dimension in dimension_to_metric_dict:
        for metric in dimension_to_metric_dict[dimension]:
            if metric in user_score.columns:
                df_final_score[dimension] += df_final_score[metric] * metric_weight_dict[metric]

    df_final_score["total_score"] = np.zeros(len(df_final_score))
    for dimension in dimension_to_metric_dict:
        df_final_score["total_score"] += df_final_score[dimension] * dimension_weight_dict[dimension]

    df_final_score = df_final_score.sort_values(by=["total_score"], ascending=False)
    df_final_score['rank'] = df_final_score['total_score'].rank(method='min',ascending=False)
    df_final_score = df_final_score[['uid','proj_quality','proj_engagement','proj_outcome','perceived_capacity','perceived_participation',
                       'total_score','rank']]
    df_final_score.columns = ['uid','quality','engagement','outcome','capacity','participation','score','rank']
    
    return df_final_score

if __name__ == "__main__":
    config_file_path = "input_files.conf"
    #read all dataframes into a dataframe of dataframe
    all_dfs = read_all_inputs(config_file_path)
    df_competitors, df_competition = get_competitors_info(all_dfs['df_competition'])
    df_users = add_competition_2021(all_dfs['df_users'], df_competition, df_competitors)
    df_users = add_im_data(df_users, all_dfs['df_evbr'])
    df_users = add_certification(df_users, all_dfs['df_cert'])
    df_users = add_rest(all_dfs['df_sql_expert'],all_dfs['df_sql_pi'],all_dfs['df_github'],all_dfs['df_imp_detail'],all_dfs['df_summary_bool'],df_users)
    df_scoring_constants = get_Scoring_Constants(all_dfs['df_scoring_constants'])
    df_final_score = score_calc(df_scoring_constants, all_dfs['weights'], df_users)
    ### save the data as a csv file with the name of current date
    today = str(datetime.date(datetime.now())).replace('-','_')
    df_final_score.to_csv(f'daily_user_score_report_{today}.csv',index=False)
    print(f'daily_user_score_report_{today} has been successfully created!')
     
df_final_score.head()

daily_user_score_report_2022_04_22 has been successfully created!


,uid,quality,engagement,outcome,capacity,participation,score,rank
4957,678,99.13124,20.00000,39.917340,18.248190,30.540844,40.569964,1.0
2755,488,55.46828,16.60725,73.274514,49.247930,25.744932,40.277775,2.0
2937,679,50.88815,4.84922,50.643214,30.022298,36.244335,33.190548,3.0
2333,2985,76.43688,9.99678,0.000000,34.802788,38.052800,30.988984,4.0
5772,246,65.10512,18.86841,1.058412,41.456600,28.443961,29.376955,5.0
